# Interact with Doctor in Training
## Role: Health Education England

## Before running through this notebook you should run through the following notebook - [Part 4.1](http://localhost:8891/notebooks/Part%204.1%20-%20Initialising%20the%20HEE%20Agent.ipynb).

## Or alternatively you can start at the beginning of the Doctors in Training flow [here](http://localhost:8889/lab/tree/Part%201%20-%20Getting%20Started.ipynb).



## 1. Initialise a controller for Health Education England

In [ ]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://hee-agent:8041"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE)
    

## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 3. Create a connection invitation to scan with the Mobile SSI Wallet

To make a connection with an identity holders wallet we must create a connection invitation and present the QR code to be scanned by the identity holders wallet. The identity holder scans this QR code and then can communicate with the agent through the exposed ngrok endpoint.

Once a connection is in an active state, the agent and the identity holders mobile wallet now have a secure pairwise relationship over which they can exchange verifiable credentials.

You must have a ssi wallet installed on your phone and connected to the Sovrin StagingNet for this to work.

### 3.1 Generate an invitation

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']

### 3.2 Display as QrCode to be Scanned by SSI Wallet

In [ ]:
import qrcode
# Link for connection invitation
input_data = inviteURL
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)

qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

### 3.3 Check the connection is in the active state

In [ ]:
import time

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'active':
#     print('ConnectionId {0} is not in active state yet'.format(connection_id))
    print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
    trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
#     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
    print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
    time.sleep(5)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


## 4. Request Passport Details Presentation

## 4a. Generate proof request

In [ ]:
print("Request proof of Passport information")

#Set some variables

REVOCATION = False
SELF_ATTESTED = False
EXCHANGE_TRACING = False

req_attrs = [
    {"name": "PSS Given Names", "restrictions": []},
    {"name": "PSS Surname", "restrictions": []},
]

if REVOCATION:
    req_attrs.append(
        {
            "name": "skill",
            "restrictions": [{"schema_id": schema_id}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "country"},)

#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Passport Credential",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if REVOCATION:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = EXCHANGE_TRACING
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": EXCHANGE_TRACING,
}

### 4b. Send the proof request to Doctor

Doctor is identified through the connection_id

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

## 4c. Verify proof presentation and populate credential template

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger. It is a bit of a big complicated object, so we show the common pattern for breaking it down, checking it's verified and accessing the data that has been presented.

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

# print proof request verification status
print('Proof request state verified?: {} '.format(verify['state'] == 'verified'))


In [ ]:
# access the revealed attributes and populate credential
credential_attributes = []
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print('Atribute name: {} Value: {}'.format(name[6:-5],val['raw']))
    credential_attributes.append({"name":name[6:-5], "value": val['raw']})
    
# set credential attribute field names to match credential
credential_attributes[0]['name'] = "RTW Given Names"
credential_attributes[1]['name'] = "RTW Surname"

In [ ]:
print(credential_attributes)

## 5 Issue Right to Work Credential

### 5.1 Retrieve stored `cred_def_id` and `schema_id`s

In [ ]:
# load all globally stored variables
%store -r training_program_schema_id
%store -r training_program_cred_def_id
%store -r right_to_work_schema_id
%store -r right_to_work_cred_def_id
%store -r training_placement_schema_id
%store -r training_placement_cred_def_id

### 5.1 Fetch the credential definition for the schema


In [ ]:
# response = await agent_controller.wallet.get_public_did()
# print(response)
# hee_public_did = response["result"]["did"]
# definitions = await agent_controller.definitions.search_created(issuer_did=hee_public_did, schema_id=right_to_work_schema_id)

# right_to_work_cred_def_id = None
# cred_def_ids = definitions['credential_definition_ids']
# if len(cred_def_ids) == 0:
#     print("You have not written a credential definition to the ledger for this public DID")
# else:
#     right_to_work_cred_def_id = cred_def_ids[0]
#     print(right_to_work_cred_def_id)
    


### 5.2 Populate the Mandatory Training Credential Attributes

In [ ]:
residency = input("Please enter residency status:")
visa = input("Please enter the VISA type :")
visa_exp = input("Please enter VISA Expiry date : ")

credential_attributes.append({"name":"RTW Residency Status", "value": residency})
credential_attributes.append({"name":"RTW Visa Type", "value": visa})
credential_attributes.append({"name":"RTW Visa Expiry Date", "value": visa_exp})
print(credential_attributes)


### 5.3 Issue the Mandatory Training Credential 

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, right_to_work_schema_id, right_to_work_cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

## 6 Issue a Training Program Credential

### 6.1 Fetch the associated credential definition id written to the Ledger in Part 4.1

This will fail if a credential definition has not been written to the ledger by the HEE agent's public DID.

In [ ]:
# response = await agent_controller.wallet.get_public_did()
# print(response)
# hee_public_did = response["result"]["did"]
# training_program_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme:0.0.1"
# definitions = await agent_controller.definitions.search_created(issuer_did=hee_public_did, schema_id=training_program_schema_id)

# training_program_cred_def_id = None
# cred_def_ids = definitions['credential_definition_ids']
# if len(cred_def_ids) == 0:
#     print("You have not written a credential definition to the ledger for this public DID")
# else:
#     training_program_cred_def_id = cred_def_ids[0]
#     print(training_program_cred_def_id)
    


### 6.2 Populate the Training Program attributes to Issue to Doctor

This would typically be done through a face to face identity check.

The notebook will ask you to input all the relevant fields.

In [ ]:
lead_employee = input("Please enter the doctors lead employee: ")
clinical_supervisor=input("Please enter the doctors clinical supervisor: ")
academic_supervisor=input("Please enter the doctors academic supervisor: ")
program_name = input("Please enter the name of the training program: ")
program_year=input("Please enter the current year of the program: ")
start_date=input("Please enter the programs start date: ")
scheduled_end = input("Please enter the scheduled end date: ")
extended_end=input("Please enter the extended_end_date: ")

credential_attributes = [
    {"name": "TPR Lead Employer", "value": lead_employee},
    {"name": "TPR Clinical Supervisor", "value": clinical_supervisor},
    {"name": "TPR Academic Supervisor", "value": academic_supervisor},
    {"name": "TPR Training Programme Name", "value": program_name},
    {"name": "TPR Programme Current Year", "value": program_year},
    {"name": "TPR Start Date", "value": start_date},
    {"name": "TPR Scheduled End Date", "value": scheduled_end},
    {"name": "TPR Extended End Date", "value": extended_end},

]




print(credential_attributes)

## 6.3 Issue Training Program Credential

This sends a credential to the doctor's mobile wallet you just connected with.

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, training_program_schema_id, training_program_cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


## 7. Issue Training Placement Credential

### 7.1 Fetch the relevant credential definition

In [ ]:

# response = await agent_controller.wallet.get_public_did()
# hee_public_did = response["result"]["did"]
# training_placement_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Training Programme Placement:0.0.1"
# definitions = await agent_controller.definitions.search_created(issuer_did=hee_public_did, schema_id=training_placement_schema_id)

# training_placement_cred_def_id = None
# cred_def_ids = definitions['credential_definition_ids']
# if len(cred_def_ids) == 0:
#     print("You have not written a credential definition to the ledger for this public DID")
# else:
#     training_placement_cred_def_id = cred_def_ids[0]
#     print(training_placement_cred_def_id)
    

### 7.2 Populate the Training Placement Credential Attributes

In [ ]:
placement_name = input("Please enter the name of the placement: ")
employer=input("Please enter the placement employer: ")
location=input("Please enter the location of the placement: ")
start_date = input("Please enter the placement start date: ")
sched_end_date =input("Please enter the scheduled end of the placement: ")
extended_end_date=input("Please enter the extended end date: ")
actual_end = input("Please enter the actual end date: ")
position_id =input("Please enter the placements position id: ")

credential_attributes = [
     {"name":"TPP Placement Name", "value": placement_name},
     {"name":"TPP Placement Employer", "value": employer},
     {"name":"TPP Placement Location", "value": location},
     {"name":"TPP Placement StartDate", "value": start_date},
     {"name":"TPP Placement Sched End Date", "value": sched_end_date},
     {"name":"TPP Placement Ext. End Date", "value": extended_end_date},
     {"name":"TPP Placement Act End Date", "value": actual_end},
     {"name":"TPP PositionId", "value": position_id}
]
print(credential_attributes)

### 7.3 Issue the Training Placement Credential

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, training_placement_schema_id, training_placement_cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

## 8. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# You can now proceed to interact with the [Lead Employer](http://127.0.0.1:8892)

Find the notebook access token by running `docker logs  interopen-hack_lead-employer-notebook_1`
